# Run pathways in FaIR

The pathways are generated elsewhere, imported here and then run.

In [1]:
import json
from multiprocessing import Pool
import platform

from climateforcing.utils import mkdir_p
import fair
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open('../data_input/fair-1.6.2-ar6/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [3]:
emissions_in = {}
results_out = {}
WORKERS = 3  # set this based on your individual machine - allows parallelisation. nprocessors-1 is a sensible shout.

In [4]:
scenarios = ["ssp245_constant-2020-ch4", "ch4_30", "ch4_40", "ch4_50", "coal-phase-out"]

In [5]:
for scenario in scenarios:
    emissions_in[scenario] = np.loadtxt('../data_output/fair_emissions_files/{}.csv'.format(scenario), delimiter=',')

## convenience function for running FaIR config with each emission species

In [6]:
def run_fair(args):
    thisC, thisF, thisT, _, thisOHU, _, thisAF = fair.forward.fair_scm(**args)
    return (thisC[:,0], thisC[:,1], thisT, np.sum(thisF, axis=1))

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
    #    updated_config[i]["scale"][43] = 0.6
        updated_config[i]["F_solar"][270:] = 0
        
    # multiprocessing is not working for me on Windows
    if platform.system() == 'Windows':
        shape = (361, len(updated_config))
        c_co2 = np.ones(shape) * np.nan
        c_ch4 = np.ones(shape) * np.nan
        t = np.ones(shape) * np.nan
        f_tot = np.ones(shape) * np.nan
        for i, cfg in tqdm(enumerate(updated_config), total=len(updated_config), position=0, leave=True):
            c_co2[:,i], c_ch4[:,i], t[:,i], f_tot[:,i] = run_fair(updated_config[i])
    
    else:
        if __name__ == '__main__':
            with Pool(WORKERS) as pool:
                result = list(tqdm(pool.imap(run_fair, updated_config), total=len(updated_config), position=0, leave=True))

        result_t = np.array(result).transpose(1,2,0)
        c_co2, c_ch4, t, f_tot = result_t
    temp_rebase = t - t[100:151,:].mean(axis=0)
    
    return c_co2, c_ch4, temp_rebase, f_tot

## Do the runs

In [7]:
for scenario in tqdm(scenarios, position=0, leave=True):
    results_out[scenario] = {}
    (
        results_out[scenario]['co2_concentrations'],
        results_out[scenario]['ch4_concentrations'],
        results_out[scenario]['temperatures'],
        results_out[scenario]['effective_radiative_forcing']
    ) = fair_process(emissions_in[scenario])

100%|████████████████████████████████████████████| 5/5 [14:26<00:00, 173.25s/it]


## Save temperature outputs to analyse elsewhere

In [8]:
for scenario in scenarios:
    for var in ['co2_concentrations', 'ch4_concentrations', 'temperatures', 'effective_radiative_forcing']:
        mkdir_p('../data_output/fair_{}/'.format(var))
        df_out = pd.DataFrame(results_out[scenario][var][245:351,:])
        df_out['year'] = np.arange(1995.5, 2101)
        df_out.set_index('year', inplace=True)
        df_out.to_csv('../data_output/fair_{}/{}.csv'.format(var, scenario), float_format="%6.4f")